### Imports

In [1]:
import sys
import pandas as pd
import numpy as np
import json
import re

### Pfade & Encoding

In [2]:
data_path = "./Hauptordner"
encoding = "utf-8"

### Zufallsseed für Reproduzierbarkeit

In [3]:
RANDOM_SEED = 42

### Dataset 2: champion_info – Grundchecks
- **Form & Größe**
- **Spaltennamen** und **Datentypen**
- **Erste Zeilen**
- **Zufällige Stichprobe**
- **Statistische Übersicht**
- **Fehlende Werte**
- **Duplikate**
- **Eindeutigkeit von Schlüsseln**
- **Min-/Max-Werte**
- **Wertebereiche**
- **Kardinalität**
- **Kategorische Verteilungen**
- **Auffälligkeiten markieren**
- **Konsistenz zwischen Spalten**
- **Komplett leere oder fast leere Spalten**
- **Datums-/Zeitspalten prüfen**
- **Zeichenketten in kategorischen Spalten prüfen**
- **Patch-Version prüfen**
- **Sonderzeichen in Namen prüfen**
- **Key vs. Name Schreibweise prüfen**
- **Titel auf Plausibilität prüfen**
- **Namens-Eindeutigkeit**
- **Prüfung auf Platzhalter-Champions**
- **Tags-Spalte prüfen**
- **Anzahl Tags pro Champion prüfen**
- **Ausreißer Checks**

### Form & Größe

In [4]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info.json"
df_champ1 = pd.read_json(pfad)
df_champ1.shape

(138, 3)

### Spaltennamen und Datentypen

In [5]:
df_champ1.dtypes

type       object
version    object
data       object
dtype: object

### Erstenzeilen anzeigen

In [6]:
df_champ1.head()

,type,version,data
1,champion,7.17.2,"{'title': 'the Dark Child', 'id': 1, 'key': 'A..."
2,champion,7.17.2,"{'title': 'the Berserker', 'id': 2, 'key': 'Ol..."
3,champion,7.17.2,"{'title': 'the Colossus', 'id': 3, 'key': 'Gal..."
4,champion,7.17.2,"{'title': 'the Card Master', 'id': 4, 'key': '..."
5,champion,7.17.2,"{'title': 'the Seneschal of Demacia', 'id': 5,..."


### Zufällige Stichprobe

In [7]:
df_champ1.sample(5, random_state=1)

,type,version,data
36,champion,7.17.2,"{'title': 'the Madman of Zaun', 'id': 36, 'key..."
34,champion,7.17.2,"{'title': 'the Cryophoenix', 'id': 34, 'key': ..."
89,champion,7.17.2,"{'title': 'the Radiant Dawn', 'id': 89, 'key':..."
6,champion,7.17.2,"{'title': 'the Dreadnought', 'id': 6, 'key': '..."
20,champion,7.17.2,"{'title': 'the Yeti Rider', 'id': 20, 'key': '..."


### Statische Übersicht

In [8]:
df_champ1.describe

<bound method NDFrame.describe of          type version                                               data
1    champion  7.17.2  {'title': 'the Dark Child', 'id': 1, 'key': 'A...
2    champion  7.17.2  {'title': 'the Berserker', 'id': 2, 'key': 'Ol...
3    champion  7.17.2  {'title': 'the Colossus', 'id': 3, 'key': 'Gal...
4    champion  7.17.2  {'title': 'the Card Master', 'id': 4, 'key': '...
5    champion  7.17.2  {'title': 'the Seneschal of Demacia', 'id': 5,...
..        ...     ...                                                ...
429  champion  7.17.2  {'title': 'the Spear of Vengeance', 'id': 429,...
432  champion  7.17.2  {'title': 'the Wandering Caretaker', 'id': 432...
497  champion  7.17.2  {'title': 'The Charmer', 'id': 497, 'key': 'Ra...
498  champion  7.17.2  {'title': 'the Rebel', 'id': 498, 'key': 'Xaya...
516  champion  7.17.2  {'title': 'The Fire below the Mountain', 'id':...

[138 rows x 3 columns]>

### Fehlende Werte

In [9]:
df_champ1.isnull().sum()

type       0
version    0
data       0
dtype: int64

### Duplikate überprüfen

In [10]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info.json"
df_champ1 = pd.read_json(pfad)
df_champ1 = pd.json_normalize(df_champ1['data'])
df_champ1.duplicated().sum()

np.int64(0)

### Eindeutigkeit von Schlüssel

In [11]:
df_champ1['key'].is_unique

True

### MIN-/MAX-Werte je Spalte

In [12]:
df_champ1.describe().loc[['min', 'max']]

,id
min,1.0
max,516.0


### Wertebereiche

In [13]:
df_champ1.describe()

,id
count,138.000000
mean,108.666667
std,111.363634
min,1.000000
25%,35.250000
50%,78.500000
75%,126.750000
max,516.000000


### Kardinalität der Spalten

In [14]:
df_champ1.nunique()

title    138
id       138
key      138
name     138
dtype: int64

### Kategorische Verteilungen

In [15]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info.json"
with open(pfad, encoding='utf-8') as f:
    raw = json.load(f)
champ_dict = raw['data']
champ_liste = list(champ_dict.values())

df_champ = pd.json_normalize(champ_liste)

### Auffälligkeiten markieren

In [16]:
df_champ1['auffällig_leer'] = df_champ1.isnull().any(axis=1)
df_champ1['auffällig_doppelt'] = df_champ1.duplicated(subset='id', keep=False)
df_champ1['auffällig_titel'] = df_champ1['title'].str.len() > 50
df_champ1[df_champ1[['auffällig_leer', 'auffällig_doppelt', 'auffällig_titel']].any(axis=1)]

,title,id,key,name,auffällig_leer,auffällig_doppelt,auffällig_titel


### Konsistenz zwischen Spalten

In [17]:
df_champ1['konsistenz_id_key'] = df_champ1['id'].astype(str) == df_champ1['key'].astype(str)
df_champ1[~df_champ1['konsistenz_id_key']]

,title,id,key,name,auffällig_leer,auffällig_doppelt,auffällig_titel,konsistenz_id_key
0,the Dark Child,1,Annie,Annie,False,False,False,False
1,the Berserker,2,Olaf,Olaf,False,False,False,False
2,the Colossus,3,Galio,Galio,False,False,False,False
3,the Card Master,4,TwistedFate,Twisted Fate,False,False,False,False
4,the Seneschal of Demacia,5,XinZhao,Xin Zhao,False,False,False,False
...,...,...,...,...,...,...,...,...
133,the Spear of Vengeance,429,Kalista,Kalista,False,False,False,False
134,the Wandering Caretaker,432,Bard,Bard,False,False,False,False
135,The Charmer,497,Rakan,Rakan,False,False,False,False
136,the Rebel,498,Xayah,Xayah,False,False,False,False


### Komplett leere oder fast leere Spalten

In [18]:
leere_spalten = df_champ1.columns[df_champ1.isnull().all()]
fast_leere_spalten = df_champ1.columns[df_champ1.isnull().mean() > 0.9]

print("Komplett leere Spalten:", list(leere_spalten))
print("Fast leere Spalten (>90% NaN):", list(fast_leere_spalten))

Komplett leere Spalten: []
Fast leere Spalten (>90% NaN): []


### Datums-/Zeitspalten prüfen

In [19]:
datum_spalten = df_champ1.select_dtypes(include=['datetime64', 'datetime64[ns]']).columns
if len(datum_spalten) == 0:
    print("Keine Datums-/Zeitspalten vorhanden.")
else:
    print("Datums-/Zeitspalten:", list(datum_spalten))
    for sp in datum_spalten:
        print(sp, "→ Min:", df_champ1[sp].min(), "| Max:", df_champ1[sp].max())

Keine Datums-/Zeitspalten vorhanden.


### Zeichenketten in kategorischen Spalten prüfen

In [20]:
kategorie_spalten = df_champ1.select_dtypes(include=['object']).columns
print("Gefundene Textspalten:", list(kategorie_spalten))

Gefundene Textspalten: ['title', 'key', 'name']


In [21]:
kategorie_spalten = df_champ1.select_dtypes(include=['object']).columns
if len(kategorie_spalten) == 0:
    print("Keine kategorischen (Text-)Spalten gefunden.")
else:
    for spalte in kategorie_spalten:
        print(f"Spalte: {spalte}")
        print(df_champ1[spalte].str.len().describe())
        print("-" * 40)

Spalte: title
count    138.000000
mean      17.913043
std        3.817895
min        9.000000
25%       15.000000
50%       18.000000
75%       21.000000
max       27.000000
Name: title, dtype: float64
----------------------------------------
Spalte: key
count    138.000000
mean       6.072464
std        1.928040
min        2.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       12.000000
Name: key, dtype: float64
----------------------------------------
Spalte: name
count    138.000000
mean       6.152174
std        2.021426
min        2.000000
25%        5.000000
50%        6.000000
75%        7.000000
max       12.000000
Name: name, dtype: float64
----------------------------------------


### Patch-Version

In [22]:
pfad = r"D:\Python\Projektarbeit\Hauptordner\champion_info.json"
with open(pfad, encoding='utf-8') as f:
    roh = json.load(f)
champ_dict = roh['data']
champ_liste = list(champ_dict.values())
df_champ1 = pd.json_normalize(champ_liste)
df_champ1['version'] = roh['version']

print("Verschiedene Patch-Versionen:")
print(df_champ1['version'].value_counts())

Verschiedene Patch-Versionen:
version
7.17.2    138
Name: count, dtype: int64


### Sonderzeichen in Namen

In [23]:
sonderzeichen = df_champ1['name'].apply(lambda x: re.findall(r'[^a-zA-Z0-9\s]', x))
df_champ1[sonderzeichen.apply(lambda x: len(x) > 0)]

,title,id,key,name,version
30,the Terror of the Void,31,Chogath,Cho'Gath,7.17.2
35,the Madman of Zaun,36,DrMundo,Dr. Mundo,7.17.2
81,the Mouth of the Abyss,96,KogMaw,Kog'Maw,7.17.2
100,the Voidreaver,121,Khazix,Kha'Zix,7.17.2
113,the Eye of the Void,161,Velkoz,Vel'Koz,7.17.2
131,the Void Burrower,421,RekSai,Rek'Sai,7.17.2


### Key vs. Name Schreibweise

In [24]:
df_champ1['name_lower'] = df_champ1['name'].str.lower()
df_champ1['key_lower'] = df_champ1['key'].str.lower()
df_champ1[df_champ1['name_lower'] != df_champ1['key_lower']]

,title,id,key,name,version,name_lower,key_lower
3,the Card Master,4,TwistedFate,Twisted Fate,7.17.2,twisted fate,twistedfate
4,the Seneschal of Demacia,5,XinZhao,Xin Zhao,7.17.2,xin zhao,xinzhao
10,the Wuju Bladesman,11,MasterYi,Master Yi,7.17.2,master yi,masteryi
20,the Bounty Hunter,21,MissFortune,Miss Fortune,7.17.2,miss fortune,missfortune
30,the Terror of the Void,31,Chogath,Cho'Gath,7.17.2,cho'gath,chogath
35,the Madman of Zaun,36,DrMundo,Dr. Mundo,7.17.2,dr. mundo,drmundo
54,the Exemplar of Demacia,59,JarvanIV,Jarvan IV,7.17.2,jarvan iv,jarvaniv
57,the Monkey King,62,MonkeyKing,Wukong,7.17.2,wukong,monkeyking
59,the Blind Monk,64,LeeSin,Lee Sin,7.17.2,lee sin,leesin
81,the Mouth of the Abyss,96,KogMaw,Kog'Maw,7.17.2,kog'maw,kogmaw


### Titel auf Plausibilität

In [25]:
if 'title' in df_champ1.columns:
    print("Übersicht der Titel:")
    print(df_champ1['title'].value_counts())
else:
    print("Spalte 'title' nicht vorhanden.")

Übersicht der Titel:
title
the Dark Child                 1
the Berserker                  1
the Colossus                   1
the Card Master                1
the Seneschal of Demacia       1
                              ..
the Spear of Vengeance         1
the Wandering Caretaker        1
The Charmer                    1
the Rebel                      1
The Fire below the Mountain    1
Name: count, Length: 138, dtype: int64


### Namens-Eindeutigkeit

In [26]:
doppelte_namen = df_champ1['name'].duplicated().sum()
print(f"Anzahl doppelter Namen: {doppelte_namen}")

Anzahl doppelter Namen: 0


### Platzhalter-Champions

In [27]:
df_champ1[df_champ1['id'] == -1]

,title,id,key,name,version,name_lower,key_lower


### Tags-Spalte

In [28]:
df_champ1.columns

Index(['title', 'id', 'key', 'name', 'version', 'name_lower', 'key_lower'], dtype='object')

In [29]:
# Keine Spalte mit dem Namen vorhanden.

### Anzahl Tags pro Champion

In [30]:
# Keine Spalte mit dem Namen vorhanden.

### Ausreißer Checks

In [31]:
# Alle numerischen Spalten (id, key) sind nur Champion-IDs bzw. Schlüsselwerte. 
# Diese sind kategorial und liefern bei Statistikmethoden nur Schein-Ausreißer. 
# Daher keine echten Ausreißer vorhanden

### Dataset 2: champion_info – Spezifische Checks
- **Sonderzeichen im Namen**
- **Key-Format-Prüfung (id, key, name)**
- **Leere Strings prüfen**
- **Doppelte id oder key**
- **Gleicher Name für mehrere ids**
- **Negativwerte in id prüfen**
- **Spalten name_lower und key_lower logisch korrekt**

### Sonderzeichen im Namen

In [32]:
df_champ1[~df_champ1["name"].str.match("^[a-zA-Z0-9 ']+$")]

# Manche Champs haben sonderzeichen was aber normal ist.

,title,id,key,name,version,name_lower,key_lower
35,the Madman of Zaun,36,DrMundo,Dr. Mundo,7.17.2,dr. mundo,drmundo


### Key-Format-Prüfung

In [33]:
print(df_champ1[["id", "key", "name"]].dtypes)

id       int64
key     object
name    object
dtype: object


### Leere Strings

In [34]:
print("Leere Werte in 'title':", (df_champ1["title"] == "").sum())
print("Leere Werte in 'name':", (df_champ1["name"] == "").sum())
print("Leere Werte in 'key':", (df_champ1["key"] == "").sum())

Leere Werte in 'title': 0
Leere Werte in 'name': 0
Leere Werte in 'key': 0


### Doppelte id oder key

In [35]:
print("Doppelte 'id':", df_champ1["id"].duplicated().sum())
print("Doppelte 'key':", df_champ1["key"].duplicated().sum())

Doppelte 'id': 0
Doppelte 'key': 0


### Gleicher Name für mehrere ids

In [36]:
mehrfach_namen = df_champ1.groupby("name")["id"].nunique()
problem_namen = mehrfach_namen[mehrfach_namen > 1]

print("Anzahl problematischer Namen:", problem_namen.shape[0])
print(problem_namen)

Anzahl problematischer Namen: 0
Series([], Name: id, dtype: int64)


### Negativwerte in id prüfen

In [37]:
df_champ1[df_champ1["id"] < 0]

,title,id,key,name,version,name_lower,key_lower


### Spalten name_lower und key_lower logisch korrekt

In [38]:
print("Abweichungen bei name_lower:", (df_champ1["name"].str.lower() != df_champ1["name_lower"]).sum())
print("Abweichungen bei key_lower:", (df_champ1["key"].str.lower() != df_champ1["key_lower"]).sum())

Abweichungen bei name_lower: 0
Abweichungen bei key_lower: 0


### Cross-Dataset-Checks
- **IDs in verbindung mit Games**
- **IDs in verbindung mit Champion 2**
- **Namensvergleich mit Champion 2**
- **-1 in Champion1 vorhanden?**

### ### IDs in verbindung mit Games

In [39]:
df = pd.read_csv("games.csv")
champ1_ids = set(df_champ1["id"])
game_ids = set(df[[
    "t1_champ1id", "t1_champ2id", "t1_champ3id", "t1_champ4id", "t1_champ5id",
    "t2_champ1id", "t2_champ2id", "t2_champ3id", "t2_champ4id", "t2_champ5id"
]].values.ravel())
unbenutzte_ids = champ1_ids - game_ids

print("Champion-IDs, die in Games nie vorkommen:")
print(unbenutzte_ids)

Champion-IDs, die in Games nie vorkommen:
set()


### IDs in verbindung mit Champion 2

In [40]:
df_champ2 = pd.read_json("champion_info_2.json")["data"].apply(pd.Series)
ids_champ1 = set(df_champ1["id"])
ids_champ2 = set(df_champ2["id"])

nur_in_champ1 = ids_champ1 - ids_champ2
print("IDs nur in Champ1:", nur_in_champ1)

nur_in_champ2 = ids_champ2 - ids_champ1
print("IDs nur in Champ2:", nur_in_champ2)

IDs nur in Champ1: set()
IDs nur in Champ2: {-1}


### Namensvergleich mit Champion 2

In [41]:
duplicated_names = df_champ2["name"].duplicated(keep=False)
df_champ2[duplicated_names]

,tags,title,id,key,name


### -1 in Champion1 vorhanden?

In [42]:
print("-1 vorhanden:", "-1" in df_champ1["id"].astype(str).values)

-1 vorhanden: False


### Abspeichern und eine Copi erstellen

In [43]:
df_champ1.to_csv("champion1.0.csv", index=False)
print("Sicherheitskopie erfolgreich gespeichert: 'champion1.0.csv'")

Sicherheitskopie erfolgreich gespeichert: 'champion1.0.csv'
